## 02-Model

#### Name: Emran Sabbagh

#### Student Number: 20088634

### Setup

The first thing I did was importing some libraries and doing the setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "NASA"

COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
else:
    ROOT = "./"

DEBUG = False
SEED = 1612

In [2]:
if COLAB:
    from google.colab import drive
    if not os.path.isdir("/content/gdrive"):
        drive.mount("/content/gdrive")
        d = "/content/gdrive/MyDrive/datasets"
        if not os.path.isdir(d): os.makedirs(d)
        if not os.path.isdir(ROOT): os.makedirs(ROOT)

def makedirs(d):
    if COLAB:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
    else:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

### Modelling

I then read the train.csv and test.csv files

In [3]:
train = f'{ROOT}/output/train.csv'
test = f'{ROOT}/output/test.csv'

df_train = pd.read_csv(train)
print(df_train.shape)
df_train.head()

(5831, 19)


,Customer,Churn,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Amount_Sum,Amount_Mean,Amount_Median,Account,Start_Year
0,797197508,0,1,1,2315.0,1565,750.0,0.676,48.0,2,1,1,2,29999,4827.96,53.644000,40.20,1,2020
1,812854728,0,1,1,7645.0,2076,5569.0,0.272,58.0,2,2,2,5,29999,7709.97,102.799600,44.11,1,2020
2,768000743,0,1,2,6394.0,0,6394.0,0.000,55.0,2,2,2,2,29999,3470.06,50.290725,29.07,2,2020
3,759029725,0,1,4,1879.0,1486,393.0,0.791,37.0,2,2,2,1,29999,5202.04,59.114091,35.04,1,2021
4,764982590,0,1,4,6036.0,1409,4627.0,0.233,56.0,1,7,1,2,50001,1741.00,38.688889,25.16,5,2018


In [4]:
df_test = pd.read_csv(test)
df_test.head()

,Customer,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Amount_Sum,Amount_Mean,Amount_Median,Account,Start_Year
0,774663629,2,2,29663.0,1743,27920.0,0.059,51,1,1,2,3,50001,14638.00,157.397849,100.350,1,2020
1,720420396,1,2,2032.0,1195,837.0,0.588,45,2,3,3,3,30001,4478.97,55.987125,32.380,1,2020
2,815283379,1,1,17268.0,1197,16071.0,0.069,57,1,1,2,3,100001,4077.01,45.300111,26.880,2,2020
3,764861610,1,3,2264.0,0,2264.0,0.000,38,2,4,2,2,29999,1204.02,43.000714,24.965,6,2020
4,742798818,1,1,2192.0,1464,728.0,0.668,45,2,6,0,4,29999,4905.03,55.112697,34.660,8,2020


I then imported more libraries for the model building

In [5]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
#######################

#######################
#######################

#######################

I then decided to do a train test split on my training data so I can check for the best classifier, my plan was to use the classifier that scores tha highest score and use it to generate the predicted data which I then submit to Kaggle

I splitted df_train to df_dummy_train and df_dummy_test

In [7]:
df_dummy_train, df_dummy_test = train_test_split(df_train, stratify=df_train.Churn, train_size=0.60, random_state=SEED)
df_dummy_train.shape, df_test.shape

((3498, 19), (3542, 18))

In [8]:
df_dummy_train.head()

,Customer,Churn,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Amount_Sum,Amount_Mean,Amount_Median,Account,Start_Year
984,764965082,0,1,1,8064.0,2417,5647.0,0.300,44.0,2,3,2,3,0,4412.00,65.850746,43.27,1,2020
4161,770535837,0,1,2,6849.0,2126,4723.0,0.310,52.0,1,6,1,3,80001,1650.99,38.395116,28.11,3,2019
3671,778674209,1,1,4,34516.0,2517,31999.0,0.073,53.0,1,3,2,4,100001,2375.99,43.199818,30.22,2,2019
40,735822140,0,1,1,3176.0,1510,1666.0,0.475,35.0,1,3,1,0,100001,2074.01,43.208542,29.48,2,2020
3214,718784194,1,1,6,1628.0,275,1353.0,0.169,43.0,2,4,2,2,29999,2219.03,45.286327,29.91,4,2020


I then defined the target feature (Churn) and created 3 lists which are:
    
cat_features which holds categorical features or columns

num_features which holds numerical features

features which is cat_features list plus num_features list

In [9]:
target = "Churn"
cat_features = [c for c in df_train.select_dtypes("category").columns if c not in target]
num_features = [c for c in df_train.select_dtypes(["int","float"]).columns if c not in target]
features = cat_features + num_features

print(f"Traget: {target}")

print(f"Categorical Features: {cat_features}")
print(f"Numerical Features: {num_features}")

Traget: Churn
Categorical Features: []
Numerical Features: ['Customer', 'Card', 'Customer_Service_Calls', 'Credit_Limit', 'Total_Revolving_Balance', 'Average_Open_To_Buy', 'Average_Utilisation_Ratio', 'Age', 'Gender', 'Education', 'Marital_Status', 'Dependents', 'Income', 'Amount_Sum', 'Amount_Mean', 'Amount_Median', 'Account', 'Start_Year']


I then created X_dummy_train, y_dummy_train, X_dummy_test and y_dummy_test and used StandardScaler for scaling and standardizing data to avoid bias https://www.digitalocean.com/community/tutorials/standardscaler-function-in-python

In [10]:
ss = StandardScaler()

X_dummy_train = ss.fit_transform(df_dummy_train[features])
y_dummy_train = df_dummy_train[target].values

X_dummy_test = ss.transform(df_dummy_test[features])
y_dummy_test = df_dummy_test[target].values

I then did model fitting on multiple models to check which model gets the highest score, I later increased the value for max_depth for DT classifier and added RFC, ADA and XGB classifiers as part of model enhancement to get a better score

We can see that XGBClassifier got the best accuracy score so I decided to use it to do the Churn predictions

In [11]:
# https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/

models = {
    "NB": GaussianNB(),
    "LR": LogisticRegression(),
    "DT": DecisionTreeClassifier(),
    "DT(max_depth=8)": DecisionTreeClassifier(max_depth=8),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(),
    'RFC': RandomForestClassifier(), 
    'ADA': AdaBoostClassifier(),
    'XGB': xgb.XGBClassifier()
}

In [12]:
for name,model in models.items():
    scores = cross_val_score(model, X_dummy_train, y_dummy_train, cv=10)
    print(f"{name:20s} accuracy = {scores.mean():.3%} ± {scores.std():.3%}")

NB                   accuracy = 84.477% ± 1.851%
LR                   accuracy = 88.451% ± 1.398%
DT                   accuracy = 90.194% ± 0.676%
DT(max_depth=8)      accuracy = 91.195% ± 1.020%


C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN                  accuracy = 85.449% ± 1.527%
SVC                  accuracy = 89.737% ± 1.293%
RFC                  accuracy = 92.824% ± 0.675%
ADA                  accuracy = 92.967% ± 0.763%
XGB                  accuracy = 94.626% ± 0.806%


In [13]:
# accuracy on test data

for name,model in models.items():
    scores = cross_val_score(model, X_dummy_test, y_dummy_test, cv=10)
    print(f"{name:20s} accuracy = {scores.mean():.3%} ± {scores.std():.3%}")

NB                   accuracy = 82.856% ± 1.885%
LR                   accuracy = 89.626% ± 0.964%
DT                   accuracy = 90.183% ± 1.758%
DT(max_depth=8)      accuracy = 89.625% ± 1.644%
KNN                  accuracy = 85.127% ± 1.922%


C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

SVC                  accuracy = 88.342% ± 1.370%
RFC                  accuracy = 91.472% ± 1.130%
ADA                  accuracy = 92.328% ± 0.982%
XGB                  accuracy = 93.270% ± 1.200%


I also checked the recall score for the classifiers and XGBClassifier got the best score in that as well

In [14]:
# https://stackoverflow.com/questions/27357121/scikit-calculate-precision-and-recall-using-cross-val-score-function

for name,model in models.items():
    scores = cross_val_score(model, X_dummy_train, y_dummy_train, cv=10, scoring='recall')
    print(f"{name:20s} recall = {scores.mean():.3%} ± {scores.std():.3%}")

NB                   recall = 29.259% ± 4.722%
LR                   recall = 45.667% ± 7.072%
DT                   recall = 69.475% ± 6.061%
DT(max_depth=8)      recall = 66.758% ± 6.912%


C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN                  recall = 23.384% ± 6.250%
SVC                  recall = 41.478% ± 6.583%
RFC                  recall = 62.889% ± 5.115%
ADA                  recall = 71.108% ± 4.722%
XGB                  recall = 78.616% ± 3.965%


In [15]:
# recall on test data

for name,model in models.items():
    scores = cross_val_score(model, X_dummy_test, y_dummy_test, cv=10, scoring='recall')
    print(f"{name:20s} recall = {scores.mean():.3%} ± {scores.std():.3%}")

NB                   recall = 47.725% ± 6.017%
LR                   recall = 48.754% ± 4.313%
DT                   recall = 69.550% ± 7.144%
DT(max_depth=8)      recall = 65.736% ± 10.368%


C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Emran\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN                  recall = 24.662% ± 9.071%
SVC                  recall = 35.886% ± 6.570%
RFC                  recall = 60.533% ± 7.355%
ADA                  recall = 69.309% ± 4.532%
XGB                  recall = 71.186% ± 7.507%


In [16]:
#######################

#######################
#######################

#######################

In [17]:
df_train['Churn'].value_counts()

0    4919
1     912
Name: Churn, dtype: int64

In [18]:
df_train.shape

(5831, 19)

I then did the same process I used before for the model building in terms of creating cat_features, num_features and features lists and creating X_train, y_train and X_test and using StandardScaler

The reason I didn't create y_test is because I don't have the Churn feature in the df_test dataframe

In [19]:
target = "Churn"
cat_features = [c for c in df_train.select_dtypes("category").columns if c not in target]
num_features = [c for c in df_train.select_dtypes(["int","float"]).columns if c not in target]
features = cat_features + num_features

print(f"Traget: {target}")

print(f"Categorical Features: {cat_features}")
print(f"Numerical Features: {num_features}")
features

Traget: Churn
Categorical Features: []
Numerical Features: ['Customer', 'Card', 'Customer_Service_Calls', 'Credit_Limit', 'Total_Revolving_Balance', 'Average_Open_To_Buy', 'Average_Utilisation_Ratio', 'Age', 'Gender', 'Education', 'Marital_Status', 'Dependents', 'Income', 'Amount_Sum', 'Amount_Mean', 'Amount_Median', 'Account', 'Start_Year']


['Customer',
 'Card',
 'Customer_Service_Calls',
 'Credit_Limit',
 'Total_Revolving_Balance',
 'Average_Open_To_Buy',
 'Average_Utilisation_Ratio',
 'Age',
 'Gender',
 'Education',
 'Marital_Status',
 'Dependents',
 'Income',
 'Amount_Sum',
 'Amount_Mean',
 'Amount_Median',
 'Account',
 'Start_Year']

In [20]:
ss = StandardScaler()

X_train = ss.fit_transform(df_train[features])
y_train = df_train['Churn'].values

X_test = ss.transform(df_test[features])

In [21]:
print(list(y_train).count(0))
print(list(y_train).count(1))

4919
912


I then did model fitting using the XGBClassifier saved the predicted Churn values in y_pred

In [22]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [23]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

I then created an empty dataframe called df and gave it two columns, the Customer column from df_test and the predicted Churn values

In [24]:
df = pd.DataFrame()
df['Customer'] = df_test['Customer']

In [25]:
df['Churn'] = y_pred
df.head()

,Customer,Churn
0,774663629,0
1,720420396,0
2,815283379,0
3,764861610,0
4,742798818,0


In [26]:
df['Churn'].value_counts()

0    3020
1     522
Name: Churn, dtype: int64

I then converted Churn values from 0 and 1 to No and Yes

I then saved the df dataframe to a csv file which I then uploaded to Kaggle

In [27]:
df['Churn'] = df['Churn'].replace({1:'Yes', 0:'No'})
df.to_csv(f"{ROOT}/output/submission.csv", index= False)